In [2]:
import ROOT as r
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import colorsys
from scipy import optimize as sciopt
import os

Welcome to JupyROOT 6.18/04


In [3]:
plt.style.available

['Solarize_Light2',
 '_classic_test',
 'bmh',
 'classic',
 'dark_background',
 'fast',
 'fivethirtyeight',
 'ggplot',
 'gm2',
 'grayscale',
 'seaborn-bright',
 'seaborn-colorblind',
 'seaborn-dark-palette',
 'seaborn-dark',
 'seaborn-darkgrid',
 'seaborn-deep',
 'seaborn-muted',
 'seaborn-notebook',
 'seaborn-paper',
 'seaborn-pastel',
 'seaborn-poster',
 'seaborn-talk',
 'seaborn-ticks',
 'seaborn-white',
 'seaborn-whitegrid',
 'seaborn',
 'tableau-colorblind10']

In [4]:
plt.style.use("gm2")

### Set-Up Histograms

In [5]:
# pick the dataset to be used

#dataset = "endgameGroupA"
#f = r.TFile("../endgame/endgameGroupA10001.root")

#dataset = "endgameGroupB"
#f = r.TFile("../endgame/endgameGroupB10001.root")

#dataset = "endgame"
#f = r.TFile("../endgame/EndgameData.root")

#dataset = "9day"
#f = r.TFile("../9day/9dayPosition10001.root")

#dataset = "60h"
#f = r.TFile("../60h/60hPosition10001.root")

#dataset = "run2Pos"
#f = r.TFile("./run2Pos/run2CVertPos.root")

dataset = "highKick"
f = r.TFile("../highKick/highKickPosition.root")

In [6]:
# pick the energy cutoffs in MeV
energy_range = [1000, 2400]

In [7]:
# set up the histograms
calo_hists = []

for i in range(1, 25):
    calo = f.Get("verticalPosition/clusters" + str(i)).Clone("calo_" + str(i))
    
    calo.SetAxisRange(energy_range[0], energy_range[1], "y")
    calo_hists.append(calo.Project3D("zx"))

In [8]:
# set the working directory for this dataset and energy cut

directory = dataset + "/" + str(energy_range[0]) + "-" + str(energy_range[1]) + "/"

# uncomment for test directory so as not to screw everything up
#directory = './test/'

print(directory)

highKick/1000-2400/


In [9]:
# just a quick sanity check, that the time bins are multiples of the g-2 period, 4.365μs
calo_hists[0].GetXaxis().GetBinWidth(9)

0.044148341165883405

In [10]:
calo_hists[0].GetNbinsX()

10001

### General Plot

In [11]:
def plot_hist_stats(data2, name, mfit_lim, vfit_lim, pos_stats, var_stats):
    
    data = data2[(data2['TimeBin'] > 30) & (data2['TimeBin'] < 700)][:]
    
    mean_lim = [data['HistMean'].min() - 1, data['HistMean'].max() + 1]
    sd_lim = [data['HistVar'].min() - 3, data['HistVar'].max() + 3]
    
   
    fig, ax = plt.subplots(1, 2)
    fig.set_size_inches(18, 10)
    
    ax[0].grid(color='xkcd:light grey', linestyle='-')
    ax[1].grid(color='xkcd:light grey', linestyle='-')
    
    #-------------------------YPosition---------------------------------------
    
    fit_data = data[(data['TimeBin'] > mfit_lim[0]) & (data['TimeBin'] < mfit_lim[1])][:]
    
    ax[0].plot(data['TimeBin'], data['HistMean'], linestyle='none', marker='o',
               color='xkcd:blue', markersize=4, label='Vertical Position');
    ax[0].plot(fit_data['TimeBin'], fit_data['HistMean'], linestyle='None', marker='o',
               color='xkcd:green', markersize=2, label='Included in Slope Fit');
    ax[0].errorbar(data['TimeBin'], data['HistMean'], yerr=data['HistMeanError'],
                   color='xkcd:blue', elinewidth=1, fmt='none');
    
    line = linear_func(pos_stats['a'], pos_stats['b'], mfit_lim)
    
    ax[0].plot(line[0], line[1], color='xkcd:green', linewidth=2, label='p0*t + p1');
    ax[0].plot(data['TimeBin'], data['HistMean'], linestyle='none', markersize=0,
               color='xkcd:green', label='p0: {0:.4e} ± {1:.4e}'.format(pos_stats['a'], pos_stats['a error']));
    ax[0].plot(data['TimeBin'], data['HistMean'], linestyle='none', markersize=0,
               color='xkcd:green', label='p1: {0:.4e} ± {1:.4e}'.format(pos_stats['b'], pos_stats['b error']));
    ax[0].plot(data['TimeBin'], data['HistMean'], linestyle='none', markersize=0,
               label='Chi2 / dof = {0:.2f}'.format(pos_stats['chi2']))
    
    ax[0].set_ylabel('Y (mm)', fontsize=16);
    ax[0].set_xlabel('Time (μs)', fontsize=16);
    ax[0].legend(loc=1, fontsize='large', labelspacing=1.5, framealpha=1);
    ax[0].set_ylim(mean_lim[0], mean_lim[1]);
    ax[0].set_title('Vertical Position', fontsize=20);
    
    #-------------------------Variance---------------------------------------
    
    fit_data = data[(data['TimeBin'] > vfit_lim[0]) & (data['TimeBin'] < vfit_lim[1])][:]
    
    ax[1].plot(data['TimeBin'], data['HistVar'], linestyle='none', marker='o',
                  color='xkcd:blue', markersize=4, label='Variance');
    ax[1].plot(fit_data['TimeBin'], fit_data['HistVar'], linestyle='None', marker='o',
                  color='xkcd:green', markersize=2, label='Included in Slope Fit');
    ax[1].errorbar(data['TimeBin'], data['HistVar'], yerr=data['HistVarError'],
                      color='xkcd:blue', elinewidth=1, fmt='none')
    
    line = linear_func(var_stats['a'], var_stats['b'], vfit_lim)

    ax[1].plot(line[0], line[1], color='xkcd:green', linewidth=2, label='p0*t + p1');
    ax[1].plot(data['TimeBin'], data['HistVar'], linestyle='none', markersize=0, 
               color='xkcd:green', label='p0: {0:.4e} ± {1:.4e}'.format(var_stats['a'], var_stats['a error']));
    ax[1].plot(data['TimeBin'], data['HistVar'], linestyle='none', markersize=0, 
               color='xkcd:green', label='p1: {0:.4e} ± {1:.4e}'.format(var_stats['b'], var_stats['b error']));
    ax[1].plot(data['TimeBin'], data['HistVar'], linestyle='none', markersize=0, 
               label='Chi2 / dof = {0:.2f}'.format(var_stats['chi2']))
    
    ax[1].set_ylabel('Variance (mm^2)', fontsize=16);
    ax[1].set_xlabel('Time (μs)', fontsize=16);
    ax[1].legend(loc=0, fontsize='large', labelspacing=1.5, framealpha=1);
    ax[1].set_ylim(sd_lim[0], sd_lim[1]);
    ax[1].set_title('Variance', fontsize=20);
    
    #-------------------------Save Figure---------------------------------------

    fig.suptitle(name, fontsize=20, y=1.05)
    
    fig.tight_layout()
    plt.savefig(directory + name + '.png', bbox_inches='tight');
    plt.close()
    
    return pos_stats, var_stats

In [12]:
def plot_varslope_vs_yslope():
    """
    Make a scatter plot of the slope in y vs the slope in variance
    """
    
    pos_data = pd.read_csv(directory + "yposition_data.csv")
    var_data = pd.read_csv(directory + "variance_data.csv")
    
    fig, ax = plt.subplots(1, 1)
    fig.set_size_inches(11, 11)
    ax.grid(color='xkcd:light grey', linestyle='-')
    
    ax.plot(pos_data['a'], var_data['a'], linestyle='none',
            marker='o', color='xkcd:blue', markersize=4);
    
    ax.errorbar(pos_data['a'], var_data['a'],
                xerr=pos_data['a error'], yerr=var_data['a error'],
                elinewidth=1, fmt='none');
    
    for i in range (0, len(pos_data)):
        label = str(int(pos_data['caloNum'].loc[i]))
        
        ax.annotate(label, (pos_data['a'].loc[i], var_data['a'].loc[i]),
                   textcoords="offset points", # how to position the text
                   xytext=(10,10), # distance from text to points (x,y)
                   ha='center') # horizontal alignment can be left, right or center)
    
    ax.set_ylabel("Variance Slope (mm^2/μs)", fontsize=16);
    ax.set_xlabel("Y Position Slope (mm/μs)", fontsize=16);
    ax.set_title("Slope in Variance vs Slope in Y", fontsize=20);
    
    fig.tight_layout();
    plt.savefig(directory + 'VarSlope_vs_YSlope.png', bbox_inches='tight');
    plt.close();

### Mean Y

In [13]:
def plot_mean_per_calo():
    """
    
    """
    
    data = pd.read_csv(directory + "yposition_data.csv")
        
    fig, ax = plt.subplots(1, 1)
    fig.set_size_inches(11, 11)
    ax.grid(color='xkcd:light grey', linestyle='-')
    
    ax.plot(data['caloNum'], data['a'], linestyle='none', marker='o',
            color='xkcd:blue', markersize=4, label='Slope of Variance vs T');
    ax.errorbar(data['caloNum'], data['a'], yerr=data['a error'],
                color='xkcd:blue', elinewidth=1, fmt='none');
    
    ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
    
    ax.set_ylabel('Slope of Y vs Time (mm/μs)', fontsize=16);
    ax.set_xlabel('Calorimeter Number', fontsize=16);
    ax.set_title('Vertical Position Slopes for 30<t<200μs and ' + str(energy_range[0]/1000) + \
                 '<E<' + str(energy_range[1]/1000) + 'GeV', fontsize=20)
    
    fig.tight_layout();
    plt.savefig(directory + 'y_slopes.png', bbox_inches='tight');
    plt.close();

### Variance

In [14]:
def plot_width_per_calo():
    """
    
    """

    data = pd.read_csv(directory + "variance_data.csv")
        
    fig, ax = plt.subplots(1, 1)
    fig.set_size_inches(11, 11)
    ax.grid(color='xkcd:light grey', linestyle='-')
    
    ax.plot(data['caloNum'], data['a'], linestyle='none', marker='o',
            color='xkcd:blue', markersize=4, label='Slope of Variance vs T');
    ax.errorbar(data['caloNum'], data['a'], yerr=data['a error'],
                color='xkcd:blue', elinewidth=1, fmt='none');
    
    ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
    
    ax.set_ylabel('Slope of Variance vs Time (mm^2/μs)', fontsize=16);
    ax.set_xlabel('Calorimeter Number', fontsize=16);
    ax.set_title('Variance Slopes for 30<t<200μs and ' + str(energy_range[0]/1000) + \
                 '<E<' + str(energy_range[1]/1000) + 'GeV', fontsize=20)
    
    fig.tight_layout();
    plt.savefig(directory + 'var_slopes.png', bbox_inches='tight');
    plt.close();

In [15]:
def var_fft(data, name):
    """
    
    """
    sp = np.fft.fft(data['HistVar'])
    freq = np.fft.fftfreq(len(data['TimeBin']), (data['TimeBin'][2] - data['TimeBin'][1]) * 10**(-6))
    
    fig, ax = plt.subplots(1, 1)
    fig.set_size_inches(11, 11)
    ax.grid(color='xkcd:light grey', linestyle='-')
    
    ax.plot(np.abs(freq) / (10**(6)), np.log10(np.abs(sp)));
    ax.set_xlim(-0.1, 1)
    
    ax.set_ylabel('$Log_{10}(sp)$', fontsize=16)
    ax.set_xlabel('Frequency (MHz)', fontsize=16)
    ax.set_title(name, fontsize=20)
    
    fig.tight_layout();
    plt.savefig(directory + 'variance_fft_' + name + '.png', bbox_inches='tight');
    plt.close();
    
    return freq, sp

In [16]:
def plot_var_fit_ends():
    """
    make a plot of variance slope and variance slope error vs fit limit end for each calo
    """
    dataframes = []
    limits = []
    for limit in range(100, 301, 10):
        df = pd.read_csv(directory + "variance_data_" + str(limit) + ".csv")
        dataframes.append(df)
        limits.append(limit)
    
    for caloIndex in range(0, 24):    # caloNum = 1 + caloIndex
        slopes = []
        slope_errors = []
        
        for df in dataframes:
            slopes.append(df.loc[caloIndex]['a'])
            slope_errors.append(df.loc[caloIndex]['a error'])
            
        num = caloIndex + 1
        if (num < 9):
            name = 'calo_0' + str(num)
        else:
            name = 'calo_' + str(num)
    
        fig, ax = plt.subplots(1,2)
        fig.set_size_inches(20,10)
        ax[0].grid(color='xkcd:light grey', linestyle='-')
        ax[1].grid(color='xkcd:light grey', linestyle='-')
        
        ax[0].plot(limits, slopes, linestyle='none', marker='o');
        ax[0].set_xlabel("Upper Fit Limit (μs)", fontsize=16)
        ax[0].set_ylabel("Variance Slope (mm^2/μs)", fontsize=16)
        ax[1].plot(limits, slope_errors, linestyle='none', marker='o');
        ax[1].set_xlabel("Upper Fit Limit (μs)", fontsize=16)
        ax[1].set_ylabel("Variance Slope Error (mm^2/μs)", fontsize=16)
        
        fig.tight_layout()
        plt.savefig(directory + "var_lims_" + name + ".png")
        plt.close()


### Calculation Helper Functions

In [17]:
def gaus_fit(hist, step=1):
    """
    Performs a Gaussian fit on the distribution of Y hits for a time slice
    
    hist: [TH2D] a histogram containing the y(t) data
    step: [int] the size of each bin for fitting
    no_spikes: [bool] if spikes should be removed. Defaults to False
    """
    stats = []
    
    for index in range(0, hist.GetNbinsX()-step, step):
        a = dict()
        try:
            proj = hist.ProjectionY("_py", index, index+step)
            
            a['TimeBin']      = index*(hist.GetXaxis().GetXmax()/hist.GetNbinsX())
            a['HistMean']     = (proj.GetMean() - 3) * 25.2
            a['HistMeanError']= proj.GetMeanError() * 25.2
            a['HistSD']       = proj.GetStdDev() * 25.2
            a['HistSDError']  = proj.GetStdDevError() * 25.2
            a['HistVar']      = (proj.GetStdDev() ** 2) * (25.2 ** 2)
            a['HistVarError'] = 2 * abs(proj.GetStdDev()) * proj.GetStdDevError() * (25.2 ** 2)
            a['Npoints']      = proj.GetEntries()
            a['RMS']          = proj.GetRMS() * 25.2
            a['RMSError']     = proj.GetRMSError() * 25.2
                
            stats.append(a)
                
        except:
            continue

    data = pd.DataFrame(columns=['TimeBin', 'HistMean',
                                 'HistMeanError', 'HistSD', 
                                 'HistSDError', 'HistVar',
                                 'HistVarError','Npoints', 
                                 'RMS', 'RMSError'])
    
    for index in range(0, len(stats)):
        data.loc[index] = stats[index]
    
    return data

In [18]:
def linear_fit(data2, mfit_lim=[30, 200], vfit_lim=[30, 200]):
    """
    Perform a linear fit on the data
    """
    data = data2.copy()
    
    # convert from crystal units to mm
    """
    data['HistMean']         = (data['HistMean'] - 3) * 25.2
    data['HistMeanError']    = data['HistMeanError'] * 25.2

    data['HistVar']           = data['HistVar'] * 25.2 * 25.2
    data['HistVarError']      = data['HistVarError'] * 25.2 * 25.2
    """
    # -------------------------------------------------------------------
    # Compute Mean Parameters
    # -------------------------------------------------------------------
    
    # restrict the data for fitting to be within a predefined time range
    fit_data = data[(data['TimeBin'] > mfit_lim[0]) & (data['TimeBin'] < mfit_lim[1])][:]
        
    # I do not know why, but having both cov=True and full=True does not return cov
    stats_list = np.polyfit(fit_data['TimeBin'], fit_data['HistMean'], 1,
                            full=True, w=1/fit_data['HistMeanError'])
    coeff, cov = np.polyfit(fit_data['TimeBin'], fit_data['HistMean'], 1,
                            full=False, cov=True, w=1/fit_data['HistMeanError'])
        
    line = linear_func(coeff[0], coeff[1], mfit_lim)
        
    # item [1] of stats_list is the fit residual
    chisq_dof = stats_list[1][0] / (len(fit_data['TimeBin']) - 2)
    param_errors = np.sqrt(np.diag(cov))
    
    pos_stats = [coeff[0], param_errors[0], coeff[1], param_errors[1], chisq_dof]
    
    # -------------------------------------------------------------------
    # Compute Variance Parameters
    # -------------------------------------------------------------------
    
    fit_data = data[(data['TimeBin'] > vfit_lim[0]) & (data['TimeBin'] < vfit_lim[1])][:]
    
    # I do not know why, but having both cov=True and full=True does not return cov
    stats_list = np.polyfit(fit_data['TimeBin'], fit_data['HistVar'], 1,
                            full=True, w=1/fit_data['HistVarError'])
    coeff, cov = np.polyfit(fit_data['TimeBin'], fit_data['HistVar'], 1,
                            full=False, cov=True, w=1/fit_data['HistVarError'])
        
    line = linear_func(coeff[0], coeff[1], vfit_lim)
    
    # item [1] of stats_list is the fit residual
    chisq_dof = stats_list[1][0] / (len(fit_data['TimeBin']) - 2)
    param_errors = np.sqrt(np.diag(cov))
    
    var_stats = [coeff[0], param_errors[0], coeff[1], param_errors[1], chisq_dof]
    
    return pos_stats, var_stats

In [19]:
def linear_func(coef1, coef2, fit_lim):
    """
    Makes a numpy linspace of a linear function
    """
    
    x = np.linspace(fit_lim[0], fit_lim[1], 100*(fit_lim[1]-fit_lim[0]))
    y = coef1 * x + coef2
    
    return [x, y]

In [20]:
def linear_func_scipy(x, a, b):
    return a * x + b

In [21]:
def line_and_sin(x, a, b, c, d, fit_lim):
    
    x = np.linspace(fit_lim[0], fit_lim[1], 100*(fit_lim[1]-fit_lim[0]))
    y = a * x + b * np.sin(c * x) + d
    
    return [x, y]

In [22]:
def line_and_sin_scipy(x, a, b, c, d):
    return a * x + b * np.sin(c * x) + d

In [23]:
# test fit

data = gaus_fit(calo_hists[0], 1)
fit_lim = [30, 200]
fit_data = data[(data['TimeBin'] > fit_lim[0]) & (data['TimeBin'] < fit_lim[1])][:]

## Mains

In [24]:
"""
Individual Calo plots and fitting
"""

mean_limits = [30, 200]
sd_limits = [30, 200]
step = 100

# set up a dataframe to store fit parameters to later
pos_data = pd.DataFrame(columns = ['caloNum', 'a', 'a error', 'b', 'b error', 'chi2'])
var_data = pd.DataFrame(columns = ['caloNum', 'a', 'a error', 'b', 'b error', 'chi2'])

# iterate through the calos
for num in range(0, len(calo_hists)):
    
    # get the name of this calo taken care of
    caloNum = num + 1
    
    if (num < 9):
        name = 'calo_0' + str(caloNum)
    else:
        name = 'calo_' + str(caloNum)
    
    # get the histogram stats for this calo
    data = gaus_fit(calo_hists[num], step)
    
    # do a fit, and get the parameters
    pos_stats, var_stats = linear_fit(data, mfit_lim=mean_limits,
                                      vfit_lim=sd_limits)

    # save the fit parameters to a dataframe row
    pos_data.loc[num] = {'caloNum':caloNum, 'a':pos_stats[0],
                         'a error':pos_stats[1], 'b':pos_stats[2],
                         'b error':pos_stats[3], 'chi2':pos_stats[4]}
    var_data.loc[num] = {'caloNum':caloNum, 'a':var_stats[0],
                         'a error':var_stats[1], 'b':var_stats[2],
                         'b error':var_stats[3], 'chi2':var_stats[4]}
    
    # make a plot of the calo mean and variance, with the fit stuff too
    plot_hist_stats(data, name, mean_limits, sd_limits,
                    pos_data.loc[num], var_data.loc[num])
    
    
# save the fit parameter dataframe to a csv for re-use
pos_data.to_csv(directory + "yposition_data.csv")
var_data.to_csv(directory + "variance_data.csv")

In [203]:
"""
Plot fft
"""
step = 1

# iterate through the calos
for num in range(0, len(calo_hists)):
    
    # get the name of this calo taken care of
    caloNum = num + 1
    
    if (num < 9):
        name = 'calo_0' + str(caloNum)
    else:
        name = 'calo_' + str(caloNum)
        
    # get the histogram stats for this calo
    data = gaus_fit(calo_hists[num], step)
    
    var_fft(data, name)

In [204]:
data = gaus_fit(calo_hists[0], 1)

print(data['TimeBin'][2] - data['TimeBin'][1])

0.04673092690730926


In [26]:
"""
Plot the slopes from each calo
"""

fit_limits = [30, 200]
step = 1

plot_width_per_calo()
plot_mean_per_calo()

In [206]:
"""
Plot variance slope using different fit endpoints
"""
step=100
mean_limits = [30, 200]

for i in range(100, 301, 10):

    sd_limits = [30, i]

    # set up a dataframe to store fit parameters to later
    var_data = pd.DataFrame(columns = ['caloNum', 'a', 'a error', 'b', 'b error', 'chi2'])

    # iterate through the calos
    for num in range(0, len(calo_hists)):

        # get the name of this calo taken care of
        caloNum = num + 1

        if (num < 9):
            name = 'calo_0' + str(caloNum)
        else:
            name = 'calo_' + str(caloNum)

        # get the histogram stats for this calo
        data = gaus_fit(calo_hists[num], step)

        # do a fit, and get the parameters
        pos_stats, var_stats = linear_fit(data, mfit_lim=mean_limits,
                                          vfit_lim=sd_limits)

        # save the fit parameters to a dataframe row
        var_data.loc[num] = {'caloNum':caloNum, 'a':var_stats[0],
                             'a error':var_stats[1], 'b':var_stats[2],
                             'b error':var_stats[3], 'chi2':var_stats[4]}

    # save the fit parameter dataframe to a csv for re-use
    var_data.to_csv(directory + "variance_data_" + str(i) + ".csv")

In [207]:
plot_var_fit_ends()